![title](imagenes/pdf/pdf1.png)
![title](imagenes/pdf/pdf2.png)
![title](imagenes/pdf/pdf3.png)
![title](imagenes/pdf/pdf4.png)
![title](imagenes/pdf/pdf5.png)
![title](imagenes/pdf/pdf6.png)
![title](imagenes/pdf/pdf7.png)



In [1]:
import pandas
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math
from lxml import etree
import re
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

In [1]:
#IMPLEMENTACION Cantidad de votos por calificacion, promedio de estrellas, fraccion de votos de humor

import pandas
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math
from lxml import etree
import re
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Calificaciones
no_humor=0
humor_e1=0
humor_e2=0
humor_e3=0
humor_e4=0
humor_e5=0
total=0

#Vectores tomados del corpus
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]


#Se suman los votos por cada calificacion
for calificacion in vec_no_humor[:]:
    no_humor+=calificacion

for calificacion in vec_e1[:]:
    humor_e1+=calificacion
    
for calificacion in vec_e2[:]:
    humor_e2+=calificacion
    
for calificacion in vec_e3[:]:
    humor_e3+=calificacion
    
for calificacion in vec_e4[:]:
    humor_e4+=calificacion

for calificacion in vec_e5[:]:
    humor_e5+=calificacion

total=no_humor + humor_e1 + humor_e2 + humor_e3 + humor_e4 + humor_e5

promedio_estrellas_pv = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/total
promedio_estrellas_pvp = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/(total - no_humor)

fraccion_votos_humor = (total - no_humor)/total



print("Total de votos", total)
print("No humor, votos", no_humor," porcentaje ", no_humor/total*100)
print("Humor 1 estrella, votos ", humor_e1, " porcentaje ", humor_e1/total*100)
print("Humor 2 estrellas, votos ", humor_e2, " porcentaje ", humor_e2/total*100)
print("Humor 3 estrellas, votos ", humor_e3, " porcentaje ", humor_e3/total*100)
print("Humor 4 estrellas, votos ", humor_e4, " porcentaje ", humor_e4/total*100)
print("Humor 5 estrellas, votos ", humor_e5, " porcentaje ", humor_e5/total*100)
print("Promedio de estrellas PV ", promedio_estrellas_pv) 
print("Promedio de estrellas PVP ", promedio_estrellas_pvp)
print("Fracción votos de humor", fraccion_votos_humor)

import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sebastiandaloia', api_key='cMoSSDnwQHyafA8prIEv')



#Impresion de valores usando ploly
labels = ['No humor','1 Estrella','2 Estrellas','3 Estrellas', '4 Estrellas', '5 Estrellas']
values = [no_humor, humor_e1, humor_e2, humor_e3, humor_e4, humor_e5]
trace = go.Pie(labels=labels, values=values)
plotly.plotly.iplot([trace], filename='basic_pie_chart')


Total de votos 26943
No humor, votos 14131  porcentaje  52.4477600861
Humor 1 estrella, votos  2960  porcentaje  10.9861559589
Humor 2 estrellas, votos  2421  porcentaje  8.98563634339
Humor 3 estrellas, votos  3274  porcentaje  12.1515792599
Humor 4 estrellas, votos  2541  porcentaje  9.43102104443
Humor 5 estrellas, votos  1616  porcentaje  5.99784730728


![title](imagenes/pdf/pdf8.png)

In [2]:
#IMPLEMENTACION Cantidad de votos por tweet
#Vectores
vec_ids=corpus['id'][:]
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]

#Variables
cant_tweets=len(corpus)

print("Cantidad de tweets ", cant_tweets)

tweet_id_vot=np.zeros((cant_tweets, 2), np.int64)

contador=0
for i in range(0, cant_tweets):
    tweet_id_vot[i, 0]=vec_ids[i]
    tweet_id_vot[i,1]=vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i]


array=tweet_id_vot[:,1]
array=np.sort(array, axis=None)
last=array[len(array) - 1]

lesseq_3=0
for i in range(0, cant_tweets):
    if( array[i] > 2 ):
        break;
    lesseq_3+=1

print("Los tweets mas votados tienen ", last, "votos")
print("Los tweets con dos o menos votos forman un total de ", lesseq_3)

contador=1
print("Los chistes mas graciosos son")
for i in range(0, cant_tweets):
    if( vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] == 21 ):
        print(contador, "\n", corpus['text'][i])

plotly.tools.set_credentials_file(username='sebastiandaloia', api_key='cMoSSDnwQHyafA8prIEv')

votos_nums = tweet_id_vot[:,1]

plt.hist(votos_nums)
plt.title("Histograma de votos por tweet")
plt.xlabel("Votos")
plt.ylabel("Tweets")

fig = plt.gcf()

plotly.plotly.plot_mpl(fig, filename='mpl-basic-histogram')


Cantidad de tweets  12106
Los tweets mas votados tienen  21 votos
Los tweets con dos o menos votos forman un total de  8668
Los chistes mas graciosos son
1 
 —¿A dónde vas tan maquillada? —A una fiesta, mamá. —¿Eres el payaso? —¡MAMÁAAA! :( —JAJAJÁ, cállate y hazme reír o no vas. —Ok. :(
High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sebastiandaloia/0 or inside your plot.ly account where it is named 'mpl-basic-histogram'


'https://plot.ly/~sebastiandaloia/0'

In [2]:
#iMPLEMENTACION Fraccion de votos de humor
import pandas
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math
from lxml import etree
import re
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Calificaciones
no_humor=0
humor_e1=0
humor_e2=0
humor_e3=0
humor_e4=0
humor_e5=0
total=0

#Vectores tomados del corpus
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]


#Se suman los votos por cada calificacion
for calificacion in vec_no_humor[:]:
    no_humor+=calificacion

for calificacion in vec_e1[:]:
    humor_e1+=calificacion
    
for calificacion in vec_e2[:]:
    humor_e2+=calificacion
    
for calificacion in vec_e3[:]:
    humor_e3+=calificacion
    
for calificacion in vec_e4[:]:
    humor_e4+=calificacion

for calificacion in vec_e5[:]:
    humor_e5+=calificacion

total=no_humor + humor_e1 + humor_e2 + humor_e3 + humor_e4 + humor_e5

promedio_estrellas_pv = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/total
promedio_estrellas_pvp = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/(total - no_humor)

fraccion_votos_humor = (total - no_humor)/total

print("Fracción votos de humor", fraccion_votos_humor)

Fracción votos de humor 0.475522399139


![title](imagenes/pdf/pdf9.png)
![title](imagenes/pdf/pdf10.png)
![title](imagenes/pdf/pdf11.png)


In [3]:
#IMPLEMENTACION Promedio de estrellas
import pandas
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time
import math
from lxml import etree
import re
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Calificaciones
no_humor=0
humor_e1=0
humor_e2=0
humor_e3=0
humor_e4=0
humor_e5=0
total=0

#Vectores tomados del corpus
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]


#Se suman los votos por cada calificacion
for calificacion in vec_no_humor[:]:
    no_humor+=calificacion

for calificacion in vec_e1[:]:
    humor_e1+=calificacion
    
for calificacion in vec_e2[:]:
    humor_e2+=calificacion
    
for calificacion in vec_e3[:]:
    humor_e3+=calificacion
    
for calificacion in vec_e4[:]:
    humor_e4+=calificacion

for calificacion in vec_e5[:]:
    humor_e5+=calificacion

total=no_humor + humor_e1 + humor_e2 + humor_e3 + humor_e4 + humor_e5

promedio_estrellas_pv = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/total
promedio_estrellas_pvp = (humor_e1 + 2*humor_e2 + 3*humor_e3 + 4*humor_e4 + 5*humor_e5)/(total - no_humor)

print("Promedio de estrellas PV ", promedio_estrellas_pv) 
print("Promedio de estrellas PVP ", promedio_estrellas_pvp)

Promedio de estrellas PV  1.3312548714
Promedio de estrellas PVP  2.79956290977


In [5]:
# IMPLEMENTACION Mejores chistes:
# 1: Para ser considerados deben tener al menos 3 votos en total 1 de ellos debe ser de humor.
# 2: Entre los candidatos los mas graciosos seran los que tengan mejor promedio de estrellas y entre estos los de mas votos.
# 3: Se listan los mejores 10 chistes, esto puede cambiarse cambiando el valor de cantidad_mejores_chistes.
mejores_chistes = np.array([])
cant_votos = []
prom_estrellas_pv = []
votos_humor = []
minimo_votos = 3
cantidad_mejores_chistes = 10
#Variables
cant_tweets=len(corpus)
for i in range(0, cant_tweets):
    votos_humor.append(vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i])
    cant_votos.append(vec_no_humor[i] + votos_humor[i])
    prom_estrellas_pv.append((vec_e1[i] + 2*vec_e2[i] + 3*vec_e3[i] + 4*vec_e4[i] + 5*vec_e5[i])/cant_votos[i])
    
    #debe tener la cantidad de votos necesaria y almenos un voto de humor para ser considerado
    if( cant_votos[i] > minimo_votos and votos_humor[i] > 0 ): 
        #solo se mostraran los 5 mejores chistes.
        chiste = np.array([corpus['text'][i], prom_estrellas_pv[i], cant_votos[i]])
        if (len(mejores_chistes) > 0):
            mejores_chistes = np.vstack((mejores_chistes, chiste))
        else :
            mejores_chistes = np.append(mejores_chistes, chiste)
   
print("")
print("Los chistes mas graciosos son: ")

ind = np.lexsort((mejores_chistes[:,2],mejores_chistes[:,1]))
for i in range(0, cantidad_mejores_chistes):
    j = len(mejores_chistes) - i - 1
    print(i+1, " - PV :",mejores_chistes[ind][j][1], "votos: ", mejores_chistes[ind][j][2])
    print("  ", mejores_chistes[j][0])


Los chistes mas graciosos son: 
1  - PV : 4.75 votos:  4
   -Mamá tuvimos un examen sorpresa en la escuela - ¿Y que pasó hijo? -Estuve muy sorprendido mamá.
2  - PV : 4.5 votos:  4
   ¿Cómo se llama la secretaria de Batman? Bati la fea... #chistes #fb
3  - PV : 4.5 votos:  4
   Quedarse atrapado en la esquina de la ducha porque el agua sale muy fría.
4  - PV : 4.4 votos:  5
   —Amor, ¿Ya está dentro? —Sí ¿Te lastima? —Sí amor, mételo despacito, duele cuando entra. —Ok, vamos a intentar otro número de zapato :)
5  - PV : 4.25 votos:  4
   —Violarte es mi sueño —¿QUÉ? —Que estudio diseño —No, ¡ahora me violas!
6  - PV : 4.25 votos:  4
   — ¡Profe! ¿Puedo ir al baño?. *Regresa con comida*.
7  - PV : 4.25 votos:  4
   Las mujeres de hoy en día con tal de no hacer limpieza en la casa, se inventan que les gusta el fútbol.
8  - PV : 4.25 votos:  4
   Siempre será mejor hablar en clases que en el recreo.
9  - PV : 4.25 votos:  4
   Lunes Martes Miércoles Jueves Casi Sábado Sábado Casi Lunes.


In [8]:
#IMPLEMENTACION Negatividad
pattern_tag = re.compile(r'lemma="(no?)"')

tam=len(corpus)
total_no=0;
total_token=0

for i in np.random.randint(0, tam, 5):
    text = corpus['text'][i]
    tweet_token=0
    token_no=0
    xml = analyzer.run(text.encode(), 'flush')
    for sentence in xml:
        for token in sentence: 
            tweet_token+=1
            m = re.search(pattern_tag, etree.tostring(token).decode())
            if m :
                token_no+=1
             
        
    total_no+=token_no    
    total_token+=tweet_token
        
print("Negación general: ", total_no/math.sqrt(total_token))

Negación general:  0.0


In [3]:
#IMPLEMENTACION cantidad de links

pattern_http = re.compile(r'http://')

num_links=0

for text in corpus['text'][:]:
    result = pattern_http.findall(text) 
    num_links+=len(result)

print("Cantidad de links encontrados ", num_links)

Cantidad de links encontrados  3280


In [26]:
#IMPLEMENTACION primera y segunda persona

pattern_tag = re.compile(r'tag="(.*?)"')

first_second_person=np.zeros((len(corpus), 2))
contador=0

tam=len(corpus)

for i in np.random.randint(0, tam, 1000):
    text = corpus['text'][i]
    total_token=0
    token_1=0
    token_2=0
    xml = analyzer.run(text.encode(), 'flush')
    for sentence in xml:
        for token in sentence:
            m = re.search(pattern_tag, etree.tostring(token).decode())
            tag=m.group(1)
            if (tag[0] == 'V'):
                if(tag[4] == '1'):
                    token_1+=1
                elif(tag[4]=='2'):
                    token_2+=1
            elif(tag[0] == 'P'):
                if(tag[2] == '1'):
                    token_1+=1
                elif(tag[2]=='2'):
                    token_2+=1
            elif(tag[0] == 'D'):
                if(tag[2] == '1'):
                    token_1+=1
                elif(tag[2]=='2'):
                    token_2+=1
            total_token+=1
    first_second_person[contador,0]=token_1/math.sqrt(total_token)
    first_second_person[contador,1]=token_2/math.sqrt(total_token)    
    contador+=1
    print ("Dato numero ",contador, "Porcentaje ", contador/len(corpus),  end="\r")
    time.sleep(1)

    
pp=[]   
sp=[]
cont=0
for t in first_second_person:
    if(cont<1000):
        pp.append(t[0])
        sp.append(t[1])    
    cont+=1    
#first_second_person    
print("Mediana primera persona ", np.median(pp))
print("Mediana segunda persona ", np.median(sp))

print("Media primera persona ", np.mean(pp))
print("Media segunda persona ", np.mean(sp))

print(pp)

print(sp)


Mediana primera persona  0.0
Mediana segunda persona  0.0
Media primera persona  0.183470578311
Media segunda persona  0.16683360628
[0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17407765595569785, 0.48507125007266594, 0.45883146774112349, 0.23570226039551587, 0.0, 0.0, 1.0606601717798212, 0.0, 0.15075567228888181, 0.75, 0.45883146774112349, 0.24253562503633297, 0.0, 0.0, 0.0, 0.0, 0.27735009811261457, 0.17677669529663687, 0.0, 0.0, 0.24253562503633297, 0.2581988897471611, 1.1470786693528088, 0.0, 0.0, 0.0, 0.3481553119113957, 0.0, 0.0, 0.0, 0.0, 0.87287156094396956, 0.18257418583505536, 0.3481553119113957, 0.0, 0.17677669529663687, 0.18257418583505536, 0.0, 0.48507125007266594, 1.2060453783110545, 0.40000000000000002, 0.41702882811414954, 0.0, 0.0, 0.0, 0.22941573387056174, 0.0, 0.0, 0.0, 1.0, 0.55470019622522915, 0.0, 1.0206207261596576, 0.40824829046386307, 0.60302268915552726, 0.16666666666666666, 0.54772255750516607, 0.0, 0.39223227027636809, 0.30151134457776363, 0.59

In [1]:
#Importacion de librerias utiles

#Libreria para analisis de datos 
#en este proyecto para leer y grabar
#archivos en csv
import pandas

#Librerias cientificas scipy, numpy, matplotlib
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

#Librerias de incluidas en python time, math
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para implementar modelos de aprendizajes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')



# Preprocesamiento

# Eliminar comentarios con menos de tres votos

In [ ]:
#Se carga el corpus de humor
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
id_f = []
text_f = []
account_id_f=[]
nh_f=[]
sh1_f=[]
sh2_f=[]
sh3_f=[]
sh4_f=[]
sh5_f=[]

#Listas del corpus
vec_id=corpus['id'][:]
vec_text=corpus['text'][:]
vec_account_id=corpus['account_id'][:]
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]

#Para cada tweet se calcula si tiene al menos tres votos, 
#en caso afirmativo se guarda su informacion, en caso
#negativo se descarta el tweet
for i in range(len(corpus)):
    if vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= 3:
        id_f.append(vec_id[i])
        text_f.append(vec_text[i])
        account_id_f.append(vec_account_id[i])
        nh_f.append(vec_no_humor[i])
        sh1_f.append(vec_e1[i])
        sh2_f.append(vec_e2[i])
        sh3_f.append(vec_e3[i])
        sh4_f.append(vec_e4[i])
        sh5_f.append(vec_e5[i])
        
#El primer filtro aplicado es guardado en el archivo corpus_filtro1.csv
d = {'id' : id_f,
    'text' : text_f,
    'account_id': account_id_f,
    'n':nh_f, 
    '1':sh1_f,
    '2':sh2_f,
    '3':sh3_f,
    '4':sh4_f,
    '5':sh5_f
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro1.csv')

## Eliminar Hashtags

In [ ]:
text_f = []

#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro1 = pandas.read_csv("corpus_filtro1.csv",encoding='utf-8')

#Generacion de patron de Hashtag
pattern_hashtag = re.compile(r'#.+?\b')


#Se sustituye el Hashtag por el string vacio, los tweets que
#van pasando por este procesamiento se van guardando en el arreglo
#text_f
for i in range(len(corpus_filtro1)):
    text_f.append(re.sub(pattern_hashtag, "", corpus_filtro1['text'][i]))
#El segundo filtro aplicado es guardado en el archivo corpus_filtro2.csv
d = {'id' : corpus_filtro1['id'][:],
    'text' : text_f,
    'account_id': corpus_filtro1['account_id'][:],
    'n':corpus_filtro1['n'][:], 
    '1':corpus_filtro1['1'][:],
    '2':corpus_filtro1['2'][:],
    '3':corpus_filtro1['3'][:],
    '4':corpus_filtro1['4'][:],
    '5':corpus_filtro1['5'][:]
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro2.csv')


# Agregar la categoria humor o no humor a los documentos

El criterio de humor o no humor esta basado en el criterio presentado en la letra del laboratorio, en donde se considerará humorístico si la mitad o más de los anotadores lo calificaron con una o más estrellas y no humorístico en caso contrario.

In [ ]:
#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro2 = pandas.read_csv("corpus_filtro2.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
text = []
category=[]
h1=[]
h2=[]
h3=[]
h4=[]
h5=[]

#Listas del corpus
vec_text=corpus_filtro2['text'][:]
vec_no_humor =corpus_filtro2['n'][:]
vec_e1 =corpus_filtro2['1'][:]
vec_e2 =corpus_filtro2['2'][:]
vec_e3 =corpus_filtro2['3'][:]
vec_e4 =corpus_filtro2['4'][:]
vec_e5 =corpus_filtro2['5'][:]

#Si hay igual o mas votos de humor que de no humor la categoria tiene el valor 1
#en caso contrario tiene el valor 0
for i in range(len(corpus_filtro2)):
    if  vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= vec_no_humor[i]:
        category.append(1)
    else:
        category.append(0)
    text.append(vec_text[i])
    h1.append(vec_e1[i])
    h2.append(vec_e2[i])
    h3.append(vec_e3[i])
    h4.append(vec_e4[i])
    h5.append(vec_e5[i])
#El tercer procesamiento aplicado sobre los tweets es guardado en el archivo corpus_filtro.3
d = {'text' : text,
    'category': category,
     '1':h1,
     '2':h2,
     '3':h3,
     '4':h4,
     '5':h5
    }
df = pandas.DataFrame(d, columns = ['text','category','1','2','3','4','5'])
df.to_csv('corpus_filtro3.csv')

## Agregar información de POS-tag 

In [ ]:
#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro3 = pandas.read_csv("corpus_filtro3.csv",encoding='utf-8')
text_list=corpus_filtro3['text'][:]

#Se genera el patron para identificar las POS de las palabras
pattern_pos = re.compile(r'pos="(.*?)"')

#Variables para datos estadisticos
num_sentences=len(corpus_filtro3)
numsentence=1

#Variable auxiliar para adherir las POS a los tweets
pos=0

#Para cada documento en el corpus de tweets que va siendo procesado
#utilizando la herramienta freeling se analizan las categorias gramaticales
#de todas las palabras que aparecen en cada tweet y esta informacion es
#adherida al texto del tweet
for d in text_list:
    if(type(d) == str):
        xml = analyzer.run(d.encode(), 'flush')
        print(numsentence, " ", math.floor( numsentence/num_sentences*100),"%", end="\r")
        for sentence in xml:        
            for token in sentence:
                token_byte=etree.tostring(token)
                m = re.search(pattern_pos, token_byte.decode())
                if m is not None:
                    text_list[ pos] = text_list[pos] + " " + m.group(1)
    pos+=1
    numsentence+=1

#El procesamiento actual del corpus es guardado en el documento corpus_filtro4.csv
d = {'text' : text_list,
    'category': corpus_filtro3['category'][:],
     '1':corpus_filtro3['1'][:],
     '2':corpus_filtro3['2'][:],
     '3':corpus_filtro3['3'][:],
     '4':corpus_filtro3['4'][:],
     '5':corpus_filtro3['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', '1','2','3','4','5'])
df.to_csv('corpus_filtro4.csv')

# Separación de datos de entrenamiento y de desarrollo

In [ ]:
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')
num_dev_tweets=math.floor(len(corpus_filtro4)*20/100)
#Se generan posiciones en el arreglo tweets al azar que representen el 20% del corpus actual
random_samples=np.random.randint(0, len(corpus_filtro4) - 1, size=num_dev_tweets)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4['text'][:]
vec_category=corpus_filtro4['category'][:]

#Se separa el 20% de corpus para desarrollo del 80% para entrenamiento
for i in range( len(corpus_filtro4)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])


#Se guardan las instancias corpus de desarrollo en corpus_filtro5_devset.csv
#y corpus de entrenamiento en corpus_filtro5_trainingset.csv
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset.csv')


d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset.csv')

## Calculo de mediana, para segundo clasificador

In [ ]:
#Se carga la instancia salvada del corpus que se encuentra en el archivo
#corpus_filtro4.csv
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')

text_list=corpus_filtro4['text'][:]
category_list=corpus_filtro4['category'][:]

h1_list=corpus_filtro4['1'][:]
h2_list=corpus_filtro4['2'][:]
h3_list=corpus_filtro4['3'][:]
h4_list=corpus_filtro4['4'][:]
h5_list=corpus_filtro4['5'][:]

medians=[]
pos=0

#Caclulo de la mediana, en values se expanden los votos de cada estrella
#de la siguiente manera si #cant_3Estrellas_en_tweet = 4, entonces se guardan
#en values 4 treses 3,3,3,3 para permitir el calculo de la mediana
#Por ultimo en la estructura medians se guarda el valor discreto (0,1,2,3,4 o 5) de la mediana
#si la cantidad de votos es par se toma el voto del medio con valor mas grande
for i in range( len(corpus_filtro4)):
    values = []
    for j in range( h1_list[i]):
        values.append(1)
    for j in range( h2_list[i] ):
        values.append(2)
    for j in range( h3_list[i] ):
        values.append(3)
    for j in range( h4_list[i] ):
        values.append(4)
    for j in range( h5_list[i] ):
        values.append(5)
    if len(values) == 0:
        values.append(0)  
    mediana=np.median(values)
    if( len(values) % 2 == 1 ):
        medians.append(math.floor(mediana))
    else:
        for i in values:
            if( i >= mediana):
                medians.append(i)
                break
    pos+=1

#El nuevo procesamiento que suma la informacion de la mediana se guarda
#en el archivo corpus_filtro4median.csv
d = {'text' : corpus_filtro4['text'][:],
    'category' : corpus_filtro4['category'][:],
    'median': medians,
    '1':corpus_filtro4['1'][:],
    '2':corpus_filtro4['2'][:],
    '3':corpus_filtro4['3'][:],
    '4':corpus_filtro4['4'][:],
    '5':corpus_filtro4['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', 'median', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro4median.csv')

## Separacion de datos de entrenamiento y desarrollo, para segundo clasificador 

In [28]:
import math
import numpy as np

#Se carga la instancia salvada en el bloque de codigo anterior
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')

num_dev_tweets=math.floor(len(corpus_filtro4median)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4median) - 1, size=num_dev_tweets)

#Listas para entrenamiento
text_train = []
median_train=[]
category_train=[]

#Listas para desarrollo
text_dev = []
median_dev=[]
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4median['text'][:]
vec_median=corpus_filtro4median['median'][:]
vec_category=corpus_filtro4median['category'][:]


for i in range( len(corpus_filtro4median)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        median_dev.append(vec_median[i])
        category_dev.append(vec_category[i])
        
    else:
        text_train.append(vec_text[i])
        median_train.append(vec_median[i])
        category_train.append(vec_category[i])

d = {'text' : text_train,
     'median' : median_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'median','category'])
df.to_csv('corpus_filtro5_trainingset_c2.csv')

d = {'text' : text_dev,
     'median': median_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'median', 'category'])
df.to_csv('corpus_filtro5_devset_c2.csv')

## Preprocesamiento, clasificador 3

In [ ]:
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
text_list=corpus_filtro4median['text'][:]
category_list=[]
median_list=corpus_filtro4median['median'][:]
h1_list=corpus_filtro4median['1'][:]
h2_list=corpus_filtro4median['2'][:]
h3_list=corpus_filtro4median['3'][:]
h4_list=corpus_filtro4median['4'][:]
h5_list=corpus_filtro4median['5'][:]
pos=0
for m in median_list:
    if(m<1):
        category_list.append(0)
    else:
        category_list.append(1)
    pos+=1
d = {'text' : text_list,
    'category': category_list,
     'median': median_list
    }
df = pandas.DataFrame(d, columns = ['text','category','median'])
df.to_csv('corpus_filtro4median_category.csv')


## Separación datos de entrenamiento y desarrollo, clasificador 3

In [ ]:
corpus_filtro4median_category = pandas.read_csv("corpus_filtro4median_category.csv",encoding='utf-8')
text=corpus_filtro4median_category['text'][:]
category=corpus_filtro4median_category['category'][:]
median=corpus_filtro4median_category['median'][:]

num_dev_tweets=math.floor(len(corpus_filtro4median_category)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4median_category) - 1, size=num_dev_tweets)

print(num_dev_tweets)
print(random_samples)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4median_category['text'][:]
vec_category=corpus_filtro4median_category['category'][:]

for i in range( len(corpus_filtro4median_category)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset_c3.csv')

d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset_c3.csv')

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import pandas

class Clasificador1:

    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_1_train()
    
    def __clasiffier_1_train(self):
        corpus_filtro5_trainingset = pandas.read_csv("corpus_filtro5_trainingset.csv",encoding='utf-8')
        text_list=corpus_filtro5_trainingset['text'][:]
        category_list=corpus_filtro5_trainingset['category'][:]
        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1
        self.train_features = self.vectorizer.fit_transform(text_list)

        self.nb_1 = MultinomialNB()
        self.nb_1.fit(self.train_features , category_list)
    def predict_clasiffier_1(self, path):       
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text'][:]
        category_devlist=corpus_filtro5_devset['category'][:]
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features = self.vectorizer.transform(text_devlist)
        return self.nb_1.predict(self.test_features)

class Clasificador2:
    
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_2_train()
        self.__median_predictor_train()

    
    def __median_predictor_train(self):
        corpus_filtro5 = pandas.read_csv("corpus_filtro5_trainingset_c2.csv",encoding='utf-8')
        text_list=corpus_filtro5['text'][:]
        median_list=corpus_filtro5['median'][:]
        self.category_list_c1=[]

        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1

        self.train_features_c1 = vectorizer.fit_transform(text_list)

        for i in median_list:
            if(i == 1):
                self.category_list_c1.append(1)
            else:
                self.category_list_c1.append(0)
        
        self.nb_median1 = MultinomialNB()
        self.nb_median1.fit(self.train_features_c1 , self.category_list_c1[:])

        self.category_list_c2=[]
        for i in median_list:
            if(i == 2):
                self.category_list_c2.append(1)
            else:
                self.category_list_c2.append(0)
        self.nb_median2 = MultinomialNB()
        self.nb_median2.fit(self.train_features_c1 , self.category_list_c2)

        self.category_list_c3=[]
        for i in median_list:
            if(i == 3):
                self.category_list_c3.append(1)
            else:
                self.category_list_c3.append(0)

        self.nb_median3 = MultinomialNB()
        self.nb_median3.fit(self.train_features_c1 , self.category_list_c3)       

        self.category_list_c4=[]
        for i in median_list:
            if(i == 4):
                self.category_list_c4.append(1)
            else:
                self.category_list_c4.append(0)

        self.nb_median4 = MultinomialNB()
        self.nb_median4.fit(self.train_features_c1 , self.category_list_c4[:])
        
        self.category_list_c5=[]
        for i in median_list:
            if(i == 5):
                self.category_list_c5.append(1)
            else:
                self.category_list_c5.append(0)
        self.nb_median5 = MultinomialNB()
        self.nb_median5.fit(self.train_features_c1 , self.category_list_c5)

        self.category_list_c0=[]
        for i in median_list:
            if(i == 0):
                self.category_list_c0.append(1)
            else:
                self.category_list_c0.append(0)
                
        self.nb_median0 = MultinomialNB()
        self.nb_median0.fit(self.train_features_c1 , self.category_list_c0)
        
    def __predict_median1_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1

        self.median_list_c1=[]
        for med in median_devlist:
            if med == 1:
                self.median_list_c1.append(1)
            else:
                self.median_list_c1.append(0)
            

        self.test_features_c1 = vectorizer.transform(text_devlist)
        return self.nb_median1.predict_proba(self.test_features_c1), self.nb_median1.predict(self.test_features_c1)
    def __predict_median2_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1

        self.median_list_c2=[]
        for med in median_devlist:
            if med == 2:
                self.median_list_c2.append(1)
            else:
                self.median_list_c2.append(0)

        
        self.test_features_c2 = vectorizer.transform(text_devlist)
        return self.nb_median2.predict_proba(self.test_features_c2), self.nb_median2.predict(self.test_features_c2)
    def __predict_median3_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1

        self.median_list_c3=[]
        for med in median_devlist:
            if med == 3:
                self.median_list_c3.append(1)
            else:
                self.median_list_c3.append(0)

        self.test_features_c3 = vectorizer.transform(text_devlist)
        return self.nb_median3.predict_proba(self.test_features_c3), self.nb_median3.predict(self.test_features_c3)
    def __predict_median4_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.median_list_c4=[]
        for med in median_devlist:
            if med == 4:
                self.median_list_c4.append(1)
            else:
                self.median_list_c4.append(0)

        self.test_features_c4 = vectorizer.transform(text_devlist)
        return self.nb_median4.predict_proba(self.test_features_c4), self.nb_median4.predict(self.test_features_c4)
    def __predict_median5_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.median_list_c5=[]
        for med in median_devlist:
            if med == 5:
                self.median_list_c5.append(1)
            else:
                self.median_list_c5.append(0)

        self.test_features_c5 = vectorizer.transform(text_devlist)
        return self.nb_median5.predict_proba(self.test_features_c5), self.nb_median5.predict(self.test_features_c5)
    def __predict_median0_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        median_devlist=corpus_filtro5_devset['median']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.median_list_c0=[]
        for med in median_devlist:
            if med == 0:
                self.median_list_c0.append(1)
            else:
                self.median_list_c0.append(0)

        self.test_features_c0 = vectorizer.transform(text_devlist)
        return self.nb_median0.predict_proba(self.test_features_c0), self.nb_median0.predict(self.test_features_c0)
    def __clasiffier_2_train(self):
        corpus_filtro5_trainingset_c2 = pandas.read_csv("corpus_filtro5_trainingset_c2.csv",encoding='utf-8')
        text_list_c2=corpus_filtro5_trainingset_c2['text'][:]
        category_list_c2=corpus_filtro5_trainingset_c2['category'][:]
        pos_c2=0
        for t in text_list_c2:
            if type(t) != str:
                text_list_c2[pos_c2]="NaN"
            pos_c2+=1
        self.train_features_class2 = self.vectorizer.fit_transform(text_list_c2)

        self.nb_2 = MultinomialNB()
        self.nb_2.fit(self.train_features_class2, category_list_c2)

    
    def predict_median(self, path):
        self.c1_pred, self.c1_predict=self.__predict_median1_c2(path)
        self.c2_pred, self.c2_predict=self.__predict_median2_c2(path)
        self.c3_pred, self.c3_predict=self.__predict_median3_c2(path)
        self.c4_pred, self.c4_predict=self.__predict_median4_c2(path)
        self.c5_pred, self.c5_predict=self.__predict_median5_c2(path)
        self.c0_pred, self.c0_predict=self.__predict_median0_c2(path)        
        predicts=[]
        proba=[]
        for i in range( len(self.c1_pred) ):
            maximum=np.max([self.c1_pred[i][1],self.c2_pred[i][1],self.c3_pred[i][1],self.c4_pred[i][1],self.c5_pred[i][1],self.c0_pred[i][1]])
            if(maximum == self.c1_pred[i][1]):
                predicts.append(1)
            elif(maximum == self.c2_pred[i][1]):
                predicts.append(2)
            elif(maximum == self.c3_pred[i][1]):
                predicts.append(3)
            elif(maximum == self.c4_pred[i][1]):
                predicts.append(4)
            elif(maximum == self.c5_pred[i][1]):
                predicts.append(5)
            elif(maximum == self.c0_pred[i][1]):
                predicts.append(0)
            proba.append(maximum)
        corpus_filtro5_devset_c2 = pandas.read_csv(path,encoding='utf-8')
        text_devlist_c2=corpus_filtro5_devset_c2['text'][:]
        category_devlist_c2=corpus_filtro5_devset_c2['category'][:]
        pos=0
        for t in text_devlist_c2:
            if type(t) != str:
                text_devlist_c2[pos]="NaN"
            pos+=1
        self.test_features_class2 = self.vectorizer.transform(text_devlist_c2)
                    
        return predicts, proba, self.nb_2.predict(self.test_features_class2)

class Clasificador3:
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_3_train()
    def __clasiffier_3_train(self):
        corpus_filtro5_trainingset_c3 = pandas.read_csv("corpus_filtro5_trainingset_c3.csv",encoding='utf-8')
        text_list=corpus_filtro5_trainingset_c3['text'][:]
        category_list=corpus_filtro5_trainingset_c3['category'][:]
        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1
        self.train_features = self.vectorizer.fit_transform(text_list)
        self.nb = MultinomialNB()
        self.nb.fit(self.train_features , category_list)
    def predict_clasiffier_1(self, path):
        corpus_filtro5_devset_c3 = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset_c3['text'][:]
        category_devlist=corpus_filtro5_devset_c3['category'][:]
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features = self.vectorizer.transform(text_devlist)
        return self.nb.predict(self.test_features)

In [17]:
clasificador = Clasificador1()
print(clasificador.predict_clasiffier_1("corpus_filtro5_devset.csv"))

[1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1
 0 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0
 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0
 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1
 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1
 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 1 1 1 1 0 0 0 0 0 0 0 1 

In [18]:
clasificador=Clasificador2()
predict, proba, humor_prob=clasificador.predict_median("corpus_filtro5_devset_c2.csv")
print(predict, "\n", proba, "\n", humor_prob)

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning: 
A value i

[3, 0, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 0, 4, 3, 3, 3, 0, 3, 0, 3, 4, 2, 3, 3, 3, 4, 3, 4, 3, 3, 3, 0, 0, 0, 3, 0, 3, 2, 0, 0, 3, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 4, 4, 0, 3, 4, 4, 4, 4, 0, 3, 3, 3, 3, 3, 0, 0, 3, 0, 4, 3, 4, 3, 3, 3, 3, 4, 0, 3, 4, 0, 0, 3, 3, 4, 0, 3, 2, 0, 0, 3, 0, 4, 3, 2, 0, 3, 3, 3, 5, 0, 4, 3, 3, 3, 0, 3, 3, 3, 0, 0, 4, 3, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 3, 0, 4, 3, 3, 2, 4, 0, 4, 3, 2, 0, 3, 3, 3, 4, 3, 0, 3, 3, 0, 0, 0, 3, 3, 4, 3, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 3, 3, 0, 3, 0, 0, 3, 4, 4, 0, 3, 0, 0, 2, 3, 3, 3, 2, 3, 0, 0, 3, 3, 3, 2, 0, 3, 3, 3, 4, 4, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 4, 3, 3, 4, 0, 0, 4, 3, 3, 4, 0, 0, 4, 3, 3, 0, 3, 4, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 2, 0, 0, 3, 0, 3, 3, 0, 4, 3, 0, 3, 3, 3, 4, 4, 3, 3, 3, 3, 0, 3, 3, 3, 0, 4, 3, 3, 3, 3, 3, 4, 2, 4, 0, 0, 3, 0, 3, 3, 0, 3, 3, 4, 3, 3, 0, 4, 0, 3, 4, 0, 3, 3, 3, 2, 0, 2, 3, 3, 4, 0, 3, 3, 0, 0, 4, 3, 0, 3, 0, 3, 3, 0, 0, 3, 4, 2, 4, 3, 0, 3, 3, 3, 0, 4, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 0, 3, 4, 0, 

In [19]:
clasificador = Clasificador3()
print(clasificador.predict_clasiffier_1("corpus_filtro5_devset_c3.csv"))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 

# Cálculo de métricas

In [28]:
corpus_filtro5_devset = pandas.read_csv("corpus_filtro5_devset.csv",encoding='utf-8')
corpus_filtro5_devset_c2 = pandas.read_csv("corpus_filtro5_devset_c2.csv",encoding='utf-8')
corpus_filtro5_devset_c3 = pandas.read_csv("corpus_filtro5_devset_c3.csv",encoding='utf-8')


#Lista de categorias del cuerpo de desarrollo
#y de categorias estimadas sobre dicho cuerpo
#para el clasificador 1
list_cat_1 = []
list_ecat_1 = []


list_cat_1=corpus_filtro5_devset['category'][:]

clasificador = Clasificador1()
list_ecat_1  = clasificador.predict_clasiffier_1("corpus_filtro5_devset.csv")

#Lista de categorias del cuerpo de desarrollo
#y de categorias estimadas sobre dicho cuerpo
#para el clasificador 2
list_cat_2 = []
list_ecat_2 = []
list_edem_2=[]


list_cat_2=corpus_filtro5_devset_c2['category'][:]
clasificador2=Clasificador2()
list_emed_2, _, list_ecat_2=clasificador2.predict_median("corpus_filtro5_devset_c2.csv")


#Lista de categorias del cuerpo de desarrollo
#y de categorias estimadas sobre dicho cuerpo
#para el clasificador 3
list_cat_3 = []
list_ecat_3 = []

list_cat_3=corpus_filtro5_devset_c3['category'][:]
clasificador3 = Clasificador3()
list_ecat_3=clasificador3.predict_clasiffier_1("corpus_filtro5_devset_c3.csv")


/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning: 
A value i

In [31]:
#Variables para clasificador 1
tp=0
fp=0
tn=0
fn=0

for i in range( len(list_cat_1) ):
    if(list_cat_1[i] == 1 and list_ecat_1[i]==1):
        tp+=1
    elif(list_cat_1[i] == 1 and list_ecat_1[i]==0):
        fn+=1
    elif(list_cat_1[i] == 0 and list_ecat_1[i]==1):
        fp+=1
    else:
        tn+=1

p = tp/(tp + fp)
r = tp / (tp + fn)
print("Medidas para el clasificador 1")
print("Precision: ", p )
print("Recall: ", r)
print("Accuracy ", (tp + fp) / (tp + fp + tn + fn))
print("E-measure ", 2*p*r/(p + r))


#Variables para el clasificador 2
tp=0
fp=0
tn=0
fn=0

for i in range( len(list_cat_2) ):
    if(list_cat_2[i] == 1 and list_ecat_2[i]==1):
        tp+=1
    elif(list_cat_2[i] == 1 and list_ecat_2[i]==0):
        fn+=1
    elif(list_cat_2[i] == 0 and list_ecat_2[i]==1):
        fp+=1
    else:
        tn+=1

p = tp/(tp + fp)
r = tp / (tp + fn)

print("Medidas para el clasificador 2")
print("Precision: ", p )
print("Recall: ", r)
print("Accuracy ", (tp + fp) / (tp + fp + tn + fn))
print("E-measure ", 2*p*r/(p + r))


#Variables para el clasificador 3
tp=0
fp=0
tn=0
fn=0

for i in range( len(list_cat_3) ):
    if(list_cat_3[i] == 1 and list_ecat_3[i]==1):
        tp+=1
    elif(list_cat_3[i] == 1 and list_ecat_3[i]==0):
        fn+=1
    elif(list_cat_3[i] == 0 and list_ecat_3[i]==1):
        fp+=1
    else:
        tn+=1

p = tp/(tp + fp)
r = tp / (tp + fn)
print("Medidas para el clasificador 3")
print("Precision: ", p )
print("Recall: ", r)
print("Accuracy ", (tp + fp) / (tp + fp + tn + fn))
print("E-measure ", 2*p*r/(p + r))


#Variables para las medianas
tp_m1=0
fp_m1=0
tn_m1=0
fn_m1=0

tp_m2=0
fp_m2=0
tn_m2=0
fn_m2=0

tp_m3=0
fp_m3=0
tn_m3=0
fn_m3=0

tp_m4=0
fp_m4=0
tn_m4=0
fn_m4=0

tp_m5=0
fp_m5=0
tn_m5=0
fn_m5=0

tp_m0=0
fp_m0=0
tn_m0=0
fn_m0=0

for i in range( len(clasificador2.median_list_c1) ):
    if(clasificador2.median_list_c1[i] == 1 and clasificador2.c1_predict[i]==1):
        tp_m1+=1
    elif(clasificador2.median_list_c1[i] == 1 and clasificador2.c1_predict[i]==0):
        fn_m1+=1
    elif(clasificador2.median_list_c1[i] == 0 and clasificador2.c1_predict[i]==1):
        fp_m1+=1
    else:
        tn_m1+=1

p_m1=-1
r_m1=-1
if(tp_m1 + fp_m1 > 0):
    p_m1 = tp_m1/(tp_m1 + fp_m1)
if(tp_m1 + fn_m1):
    r_m1 = tp_m1 / (tp_m1 + fn_m1)

print("Predicter Median 1")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m1,"\t|\t", fp_m1, "\t|")
print("system negative|\t", fn_m1, "\t|\t", tn_m1, "\t|")
if p_m1 > 0:
    print("Presicion ", p_m1)
else:
    print("Cant calculate Presicion, division by zero")
if r_m1 > 0:
    print("Recall ", r_m1)
else:
    print("Cant calculate Recall, division by zero")
if(p_m1 > 0 and r_m1 > 0):
    print("Accuracy ", (tp_m1 + fp_m1) / (tp_m1 + fp_m1 + tn_m1 + fn_m1))
    print("E-measure ", 2*p_m1*r_m1/(p_m1 + r_m1))


for i in range( len(clasificador2.median_list_c2) ):
    if(clasificador2.median_list_c2[i] == 1 and clasificador2.c2_predict[i]==1):
        tp_m2+=1
    elif(clasificador2.median_list_c2[i] == 1 and clasificador2.c2_predict[i]==0):
        fn_m2+=1
    elif(clasificador2.median_list_c2[i] == 0 and clasificador2.c2_predict[i]==1):
        fp_m2+=1
    else:
        tn_m2+=1

p_m2=-1
r_m2=-1
if(tp_m2 + fp_m2 > 0):
    p_m2 = tp_m2/(tp_m2 + fp_m2)
if(tp_m2 + fn_m2):
    r_m2 = tp_m2 / (tp_m2 + fn_m2)

print("Predicter Median 2")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m2,"\t|\t", fp_m2, "\t|")
print("system negative|\t", fn_m2, "\t|\t", tn_m2, "\t|")
if p_m2 > 0:
    print("Presicion ", p_m2)
else:
    print("Cant calculate Presicion, division by zero")
if r_m2 > 0:
    print("Recall ", r_m2)
else:
    print("Cant calculate Recall, division by zero")
if(p_m2 > 0 and r_m2 > 0):
    print("Accuracy ", (tp_m2 + fp_m2) / (tp_m2 + fp_m2 + tn_m2 + fn_m2))
    print("E-measure ", 2*p_m2*r_m2/(p_m2 + r_m2))

for i in range( len(clasificador2.median_list_c3) ):
    if(clasificador2.median_list_c3[i] == 1 and clasificador2.c3_predict[i]==1):
        tp_m3+=1
    elif(clasificador2.median_list_c3[i] == 1 and clasificador2.c3_predict[i]==0):
        fn_m3+=1
    elif(clasificador2.median_list_c3[i] == 0 and clasificador2.c3_predict[i]==1):
        fp_m3+=1
    else:
        tn_m3+=1

p_m3=-1
r_m3=-1
if(tp_m3 + fp_m3 > 0):
    p_m3 = tp_m3/(tp_m3 + fp_m3)
if(tp_m3 + fn_m3):
    r_m3 = tp_m3 / (tp_m3 + fn_m3)

print("Predicter Median 3")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m3,"\t|\t", fp_m3, "\t|")
print("system negative|\t", fn_m3, "\t|\t", tn_m3, "\t|")
if p_m3 > 0:
    print("Presicion ", p_m3)
else:
    print("Cant calculate Presicion, division by zero")
if r_m3 > 0:
    print("Recall ", r_m3)
else:
    print("Cant calculate Recall, division by zero")
if(p_m3 > 0 and r_m3 > 0):
    print("Accuracy ", (tp_m3 + fp_m3) / (tp_m3 + fp_m3 + tn_m3 + fn_m3))
    print("E-measure ", 2*p_m3*r_m3/(p_m3 + r_m3))

for i in range( len(clasificador2.median_list_c4) ):
    if(clasificador2.median_list_c4[i] == 1 and clasificador2.c4_predict[i]==1):
        tp_m4+=1
    elif(clasificador2.median_list_c4[i] == 1 and clasificador2.c4_predict[i]==0):
        fn_m4+=1
    elif(clasificador2.median_list_c4[i] == 0 and clasificador2.c4_predict[i]==1):
        fp_m4+=1
    else:
        tn_m4+=1

p_m4=-1
r_m4=-1
if(tp_m4 + fp_m4 > 0):
    p_m4 = tp_m4/(tp_m4 + fp_m4)
if(tp_m4 + fn_m4):
    r_m4 = tp_m4 / (tp_m4 + fn_m4)

print("Predicter Median 4")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m4,"\t|\t", fp_m4, "\t|")
print("system negative|\t", fn_m4, "\t|\t", tn_m4, "\t|")
if p_m4 > 0:
    print("Presicion ", p_m4)
else:
    print("Cant calculate Presicion, division by zero")
if r_m4 > 0:
    print("Recall ", r_m4)
else:
    print("Cant calculate Recall, division by zero")
if(p_m4 > 0 and r_m4 > 0):
    print("Accuracy ", (tp_m4 + fp_m4) / (tp_m4 + fp_m4 + tn_m4 + fn_m4))
    print("E-measure ", 2*p_m4*r_m4/(p_m4 + r_m4))


for i in range( len(clasificador2.median_list_c5) ):
    if(clasificador2.median_list_c5[i] == 1 and clasificador2.c5_predict[i]==1):
        tp_m5+=1
    elif(clasificador2.median_list_c5[i] == 1 and clasificador2.c5_predict[i]==0):
        fn_m5+=1
    elif(clasificador2.median_list_c5[i] == 0 and clasificador2.c5_predict[i]==1):
        fp_m5+=1
    else:
        tn_m5+=1

p_m5=-1
r_m5=-1
if(tp_m5 + fp_m5 > 0):
    p_m5 = tp_m5/(tp_m5 + fp_m5)
if(tp_m5 + fn_m5):
    r_m5 = tp_m5 / (tp_m5 + fn_m5)

print("Predicter Median 5")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m5,"\t|\t", fp_m5, "\t|")
print("system negative|\t", fn_m5, "\t|\t", tn_m5, "\t|")
if p_m5 > 0:
    print("Presicion ", p_m5)
else:
    print("Cant calculate Presicion, division by zero")
if r_m5 > 0:
    print("Recall ", r_m5)
else:
    print("Cant calculate Recall, division by zero")
if(p_m5 > 0 and r_m5 > 0):
    print("Accuracy ", (tp_m5 + fp_m5) / (tp_m5 + fp_m5 + tn_m5 + fn_m5))
    print("E-measure ", 2*p_m5*r_m5/(p_m5 + r_m5))
    
for i in range( len(clasificador2.median_list_c0) ):
    if(clasificador2.median_list_c0[i] == 1 and clasificador2.c0_predict[i]==1):
        tp_m0+=1
    elif(clasificador2.median_list_c0[i] == 1 and clasificador2.c0_predict[i]==0):
        fn_m0+=1
    elif(clasificador2.median_list_c0[i] == 0 and clasificador2.c0_predict[i]==1):
        fp_m0+=1
    else:
        tn_m0+=1

p_m0=-1
r_m0=-1
if(tp_m0 + fp_m0 > 0):
    p_m0 = tp_m0/(tp_m0 + fp_m0)
if(tp_m0 + fn_m0):
    r_m0 = tp_m0 / (tp_m0 + fn_m0)

print("Predicter Median 0")
print("\t\t gold positive  | gold negative")
print("system positive|\t", tp_m0,"\t|\t", fp_m0, "\t|")
print("system negative|\t", fn_m0, "\t|\t", tn_m0, "\t|")
if p_m0 > 0:
    print("Presicion ", p_m0)
else:
    print("Cant calculate Presicion, division by zero")
if r_m0 > 0:
    print("Recall ", r_m0)
else:
    print("Cant calculate Recall, division by zero")
if(p_m0 > 0 and r_m0 > 0):
    print("Accuracy ", (tp_m0 + fp_m0) / (tp_m0 + fp_m0 + tn_m0 + fn_m0))
    print("E-measure ", 2*p_m0*r_m0/(p_m0 + r_m0))
    
print("Confusion matrix")
print("\t 0 \t 1 \t 2 \t 3 \t 4 \t 5 ")


conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c0_predict)):
    if(clasificador2.c0_predict[pos] == 1 and clasificador2.median_list_c0[pos] == 0):
        if(medians[pos] == 1):
            conf1+=1
        if(medians[pos] == 2):
            conf2+=1
        if(medians[pos] == 3):
            conf3+=1
        if(medians[pos] == 4):
            conf4+=1
        if(medians[pos] == 5):
            conf5+=1
print("0 \t", tp_m0, "\t", conf1, "\t", conf2,"\t", conf3, "\t", conf4,"\t", conf5);

conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c1_predict)):
    if(clasificador2.c1_predict[pos] == 1 and clasificador2.median_list_c1[pos] == 0):
        if(medians[pos] == 0):
            conf0+=1
        if(medians[pos] == 2):
            conf2+=1
        if(medians[pos] == 3):
            conf3+=1
        if(medians[pos] == 4):
            conf4+=1
        if(medians[pos] == 5):
            conf5+=1
print("1 \t", conf0, "\t", tp_m1, "\t", conf2,"\t", conf3, "\t", conf4,"\t", conf5);


conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c2_predict)):
    if(clasificador2.c2_predict[pos] == 1 and clasificador2.median_list_c2[pos] == 0):
        if(medians[pos] == 0):
            conf0+=1
        if(medians[pos] == 1):
            conf1+=1
        if(medians[pos] == 3):
            conf3+=1
        if(medians[pos] == 4):
            conf4+=1
        if(medians[pos] == 5):
            conf5+=1
print("2 \t", conf0, "\t", conf1, "\t", tp_m2,"\t", conf3, "\t", conf4,"\t", conf5);

conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c3_predict)):
    if(clasificador2.c3_predict[pos] == 1 and clasificador2.median_list_c3[pos] == 0):
        if(medians[pos] == 0):
            conf0+=1
        if(medians[pos] == 1):
            conf1+=1
        if(medians[pos] == 2):
            conf2+=1
        if(medians[pos] == 4):
            conf4+=1
        if(medians[pos] == 5):
            conf5+=1
print("3 \t", conf0, "\t", conf1, "\t", conf2,"\t", tp_m3, "\t", conf4,"\t", conf5);

conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c4_predict)):
    if(clasificador2.c4_predict[pos] == 1 and clasificador2.median_list_c4[pos] == 0):
        if(medians[pos] == 0):
            conf0+=1
        if(medians[pos] == 1):
            conf1+=1
        if(medians[pos] == 2):
            conf2+=1
        if(medians[pos] == 3):
            conf3+=1
        if(medians[pos] == 5):
            conf5+=1
print("4 \t", conf0, "\t", conf1, "\t", conf2,"\t", conf3, "\t", tp_m4,"\t", conf5);

conf0=0
conf1=0
conf2=0
conf3=0
conf4=0
conf5=0


for pos in range(len(clasificador2.c5_predict)):
    if(clasificador2.c5_predict[pos] == 1 and clasificador2.median_list_c5[pos] == 0):
        if(medians[pos] == 0):
            conf0+=1
        if(medians[pos] == 1):
            conf1+=1
        if(medians[pos] == 2):
            conf2+=1
        if(medians[pos] == 3):
            conf3+=1
        if(medians[pos] == 4):
            conf4+=1
print("5 \t", conf0, "\t", conf1, "\t", conf2,"\t", conf3, "\t", conf4,"\t", tp_m5);




Medidas para el clasificador 1
Precision:  0.7427745664739884
Recall:  0.803125
Accuracy  0.5492063492063493
E-measure  0.7717717717717717
Medidas para el clasificador 2
Precision:  0.7680722891566265
Recall:  0.7634730538922155
Accuracy  0.5363489499192245
E-measure  0.7657657657657657
Medidas para el clasificador 3
Precision:  0.8198970840480274
Recall:  0.9676113360323887
Accuracy  0.9313099041533547
E-measure  0.8876508820798514
Predicter Median 1
		 gold positive  | gold negative
system positive|	 0 	|	 0 	|
system negative|	 68 	|	 551 	|
Cant calculate Presicion, division by zero
Cant calculate Recall, division by zero
Predicter Median 2
		 gold positive  | gold negative
system positive|	 2 	|	 2 	|
system negative|	 78 	|	 537 	|
Presicion  0.5
Recall  0.025
Accuracy  0.006462035541195477
E-measure  0.047619047619047616
Predicter Median 3
		 gold positive  | gold negative
system positive|	 7 	|	 17 	|
system negative|	 140 	|	 455 	|
Presicion  0.2916666666666667
Recall  0.0476